# Ingesting an Image

This example shows how to ingest a grayscale image and save the results locally.

## Getting the Data

Before we can begin with the ingest, we must first download the data from S3. This curl command will download a file from S3 and save it to your `/tmp` direcotry.

**A side note**: Files can be retrieved directly from S3 using the methods shown in this tutorial. However, this could not be done in this instance due to permission requirements needed to access the file.

In [ ]:
!curl -o /tmp/cropped.tif https://s3.amazonaws.com/geopyspark-test/example-files/cropped.tif

## The Code

With our file downloaded we can begin the ingest.

In [ ]:
import geopyspark as gps

from pyspark import SparkContext

### Setting Up the SparkContext

The first thing one needs to do when using GeoPySpark is to setup `SparkContext`. Because GeoPySpark is backed by Spark, the `pysc` is needed to initialize our starting classes.

For those that are already familiar with Spark, you may already know there are multiple ways to create a `SparkContext`. When working with GeoPySpark, it is advised to create this instance via `SparkConf`. This is where the `geopyspark_conf` comes into play. There are numerous settings for `SparkConf`, and some **have** to be set a certain way in order for GeoPySpark to work. Thus, `geopyspark_conf` was created as way for a user to set the basic parameters without having to worry about setting the other requirements.

In [ ]:
conf = gps.geopyspark_conf(master="local[*]", appName="ingest-example")
pysc = SparkContext(conf=conf)

### Reading in the Data

After the creation of `pysc`, we can now read in the data. For this example, we will be reading in a single GeoTiff that contains spatial data. Hence, why we set the `layer_type` to `LayerType.SPATIAL`.

In [ ]:
raster_layer = gps.geotiff.get(pysc=pysc, layer_type=gps.LayerType.SPATIAL, uri="file:///tmp/cropped.tif")

### Tiling the Data

With the metadata collected, it is now time to format the data within the layer to our desired layout. The aptly named, `tile_to_layout`, method will cut and arrange the rasters in the layer to the layout within the metadata; giving us a new class instance of `TiledRasterLayer`.

`tile_to_layout` tiled the rasters within the layer to certain layout. For this example, we will be tiling to a `GlobalLayout`. This means that layout will be constrained in such a way that it fits the global, CRS extent. Any layer that is going to be displayed via a TMS server **must** have a `GlobalLayout`.

In [ ]:
tiled_raster_layer = raster_layer.tile_to_layout(gps.GlobalLayout())
tiled_raster_layer

With our tiled data, we'd like to make a tile server from it and show it in on a map. Therefore, we have to make sure that the tiles within the layer are in the right projected. In this case, Web Mercator. Using the `reproject` method we can choose our target `crs` and `layout`.

In [ ]:
# crs before reprojection
tiled_raster_layer.layer_metadata.crs

In [ ]:
reprojected_layer = tiled_raster_layer.reproject(target_crs=3857,
                                                 layout=gps.GlobalLayout(zoom=tiled_raster_layer.zoom_level))

In [ ]:
# crs after reprojection
reprojected_layer.layer_metadata.crs

You may have noticed that `layout` was specified again using `GlobalLayout(zoom=tiled_raster_layer.zoom_level)`.
This was done in order to keep the layout consistent with what was created during the `tile_to_layout` step.

### Pyramiding the Data

Now it's time to pyramid! With our reprojected data, we will create an instance of `Pyramid` that contains 12 `TiledRasterLayer`s. Each one having it's own `zoom_level` from 11 to 0.

In [ ]:
pyramided_layer = reprojected_layer.pyramid()

In [ ]:
pyramided_layer.levels

### Saving the Pyramid Locally

To save all of the `TiledRasterLayer`s within `pyramid_layer`, we just have to loop through values of `pyramid_layer.level` and write each layer locally.

In [ ]:
for tiled_layer in pyramided_layer.levels.values():
    gps.write(uri="file:///tmp/ingested-image", layer_name="ingested-image", tiled_raster_rdd=tiled_layer)